In [33]:
import pandas as pd
import datetime


def time_to_seconds(time_str):
    """Convert a time string in the format H:M:S.microseconds to seconds."""
    if not time_str:  # Handle empty strings
        return None
    if time_str != time_str:
        return None
    time_obj = datetime.datetime.strptime(time_str, "%H:%M:%S.%f")
    return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1_000_000

# Read res.csv
df = pd.read_csv('res.csv')
# All columns that have an int header
int_columns = [col for col in df.columns if col.isdigit()]
# Convert all columns to seconds
df[int_columns] = df.apply(lambda row: row[int_columns].apply(time_to_seconds), axis=1)

df

,metodo,8,16,32,64
0,RPF,221.729603,522.336412,1410.031252,4080.225128
1,Vanilla,5.053546,9.297575,19.009304,41.573534


In [34]:
# Default plot
#import matplotlib.pyplot as plt
#df.plot()



# Advanced plot
import plotly.express as px
df_input = df
df_melted = df_input.melt(id_vars="metodo", var_name="n", value_name="time")
fig = px.line(
    df_melted,
    x="n",
    y="time",
    color="metodo",
    markers=True,
    labels={"size": "Size", "value": "Value", "metodo": "Method", "time": "Time (s)", "n": "Sample Per Pixel (spp)"},
    title="Comparación de tiempo de ejecución (PBRTv2)"
)

fig.update_layout(
  template="plotly_dark",
)

# Save image to ./imgs/analisis/execution_time.png
fig.write_image("./imgs/analisis/execution_time.png")

fig.show()

In [35]:
# Get list of all pngs in imgs/rpf
from typing import Union
import os
import glob
import numpy as np
from skimage.metrics import structural_similarity as ssim
pngs = glob.glob("./imgs/rpf/*.png")

# Load ground truth
import cv2
gt = cv2.imread("./imgs/rpf/highres8192.png", cv2.IMREAD_UNCHANGED)

# Score function
def image_scores(gt, img):
    s1 = cv2.PSNR(gt, img)
    s2 = ssim(gt, img, win_size=7, channel_axis=-1)  # Specify win_size and channel_axis
    s3 = np.mean((gt - img) ** 2)
    return s1, s2, s3

# Capture the {experiment}_{spp}_{method}_....png
import re
data = []
data_header = ["png", "experiment", "spp", "method", "PSNR", "SSIM", "MSE"]
data.append(data_header)
for png in pngs:

    # Load image
    img = cv2.imread(png, cv2.IMREAD_UNCHANGED)
    # Score image
    scores = image_scores(gt, img)

    match = re.search(r"(\w+)_(\d+)_(\w+)_", png)
    if match:
        experiment, spp_str, method = match.groups()
        spp = int(spp_str)

        this_data : list[Union[str, float]] = [png,]
        this_data.extend( [experiment, spp, method] )
        this_data.extend(scores)
        data.append(this_data)

# To DataFrame
df = pd.DataFrame(data[1:], columns=data[0])
df

C:\Users\Rodrigo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skimage\metrics\_structural_similarity.py:142: UserWarning:

Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.



,png,experiment,spp,method,PSNR,SSIM,MSE
0,./imgs/rpf\Conference_16_MC_0016.png,Conference,16,MC,-32.133512,0.426769,22889.252479
1,./imgs/rpf\Conference_16_RPF_flt.png,Conference,16,RPF,-31.382088,0.703229,23646.348243
2,./imgs/rpf\Conference_32_MC_0032.png,Conference,32,MC,-31.874736,0.464388,23139.351452
3,./imgs/rpf\Conference_32_RPF_flt.png,Conference,32,RPF,-31.303839,0.710534,23543.221871
4,./imgs/rpf\Conference_8_MC_0008.png,Conference,8,MC,-32.328396,0.398744,22640.810470
5,./imgs/rpf\Conference_8_RPF_flt.png,Conference,8,RPF,-31.423616,0.696625,23682.007506


In [39]:
df_input = df[["method", "spp", "MSE"]]
# Only select method="RPF"
df_input = df_input[df_input["method"] == "RPF"]
df_melted = df_input.melt(id_vars=["spp", "method"], var_name="metric", value_name="value")
df_melted = df_melted.sort_values(by=["spp", "method"])
fig = px.line(
    df_melted,
    x="spp",
    y="value",
    color="method",
    markers=True,
    labels={"size": "Size", "value": "Value", "method": "Method", "value": "Value", "spp": "Sample Per Pixel (spp)"},
    title="Comparación de MSE (PBRTv2)"
)
fig.update_layout(
  template="plotly_dark",
)
fig.write_image("./imgs/analisis/mse.png")
fig.show()
